In [51]:
import requests
import json
import pandas as pd
import io
from datetime import date
import covidcast
import geopandas as gpd
pd.set_option('display.max_rows', 500)

# New York Times Dataset

## Mask Wearing Data

In [52]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
df = pd.read_csv(StringIO(s))
print(df.shape)
print(df.columns)
df.head(100)

(3142, 6)
Index(['COUNTYFP', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS'], dtype='object')


,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
5,1011,0.031,0.040,0.144,0.286,0.500
6,1013,0.102,0.053,0.257,0.137,0.451
7,1015,0.152,0.108,0.130,0.167,0.442
8,1017,0.117,0.037,0.150,0.136,0.560
9,1019,0.135,0.027,0.161,0.158,0.520


## Geopandas File

In [53]:
data = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')
print(data.shape)
data.head(100)

(3331, 88)


,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,1,Autauga,Alabama,AL,01,01001,1.161440,7742.67,89.926440,13.8,3.6,118.9591,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4305,50,815,1026,1498,2440,2271,85,55,6,2,55601,7587,942,59338,0,0,371934,4873,0,1972204,8050,37,0,55200,41412,10475,159,568,5,41,1012,1528,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,55200,1528,22052,4711,37.0,29.0,49.0,26.0,59.0,40.0,36.0,30.0,9.0,48.0,53.0,48.0,61.0,39.0,66.545566,96.320346,100.000000,2.209382e+09,2.468399e+05,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,2,Baldwin,Alabama,AL,01,01003,1.201650,6450.73,77.515113,9.8,3.6,115.4508,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,14064,169,3949,4792,7373,11410,13141,386,362,51,8,218022,21069,3393,57588,0,0,371934,4873,0,1972204,40665,109,0,208107,172768,19529,1398,1668,9,410,2972,9353,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,208107,9353,82998,24551,109.0,132.0,222.0,209.0,220.0,210.0,137.0,117.0,42.0,145.0,200.0,166.0,203.0,125.0,49.994955,74.029401,120.089286,5.770469e+09,7.284451e+05,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,3,Barbour,Alabama,AL,01,01005,2.156863,6149.27,132.631325,30.9,5.2,68.9280,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1530,33,422,551,841,1305,1515,74,30,5,2,24881,6788,433,34382,0,0,371934,4873,0,1972204,4634,2,0,25782,11898,12199,63,85,1,86,344,1106,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,25782,1106,9864,2820,2.0,11.0,3.0,22.0,30.0,45.0,11.0,8.0,2.0,6.0,7.0,20.0,27.0,0.0,8.038262,56.711409,88.571429,3.258643e+09,3.072852e+05,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,4,Bibb,Alabama,AL,01,01007,2.444208,8401.79,205.357143,21.8,4.0,92.3478,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1882,46,427,488,624,842,1280,35,25,4,1,22400,4400,344,46064,0,0,371934,4873,0,1972204,3661,19,0,22527,16801,4974,8,37,0,0,160,547,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,22527,547,9044,2122,19.0,9.0,20.0,17.0,25.0,30.0,16.0,7.0,14.0,14.0,28.0,23.0,32.0,15.0,84.821429,19.047619,NaN,2.310715e+09,2.278870e+05,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,5,Blount,Alabama,AL,01,01009,1.327434,8205.39,108.921162,13.2,3.5,101.0645,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4746,63,866,1459,1776,2802,3330,25,25,6,2,57840,7527,878,50412,0,0,371934,4873,0,19722

In [54]:
data.columns

Index(['OBJECTID', 'Countyname', 'ST_Name', 'ST_Abbr', 'ST_ID', 'FIPS',
       'FatalityRa', 'Confirmedb', 'DeathsbyPo', 'PCTPOVALL_', 'Unemployme',
       'Med_HH_Inc', 'State_Fata', 'DateChecke', 'EM_type', 'EM_date',
       'EM_notes', 'url', 'Thumbnail', 'Confirmed', 'Deaths', 'Age_85',
       'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Beds_Licen',
       'Beds_Staff', 'Beds_ICU', 'Ventilator', 'POP_ESTIMA', 'POVALL_201',
       'Unemployed', 'Median_Hou', 'Recovered', 'Active', 'State_Conf',
       'State_Deat', 'State_Reco', 'State_Test', 'AgedPop', 'NewCases',
       'NewDeaths', 'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop',
       'AsianPop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'Wh_Alone',
       'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More',
       'Not_Hisp', 'Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_Over75',
       'Agetotal', 'NonHisp', 'Age_35_64', 'Age_65_74', 'Day_1', 'Day_2',
       'Day_3', 'Day_4', 'Day_5', 'Day_6'

In [55]:
data.Countyname.value_counts()

Unassigned         52
Washington         31
Jefferson          26
Franklin           25
Lincoln            24
                   ..
Hemphill            1
Granite             1
Camuy               1
Kenai Peninsula     1
San Joaquin         1
Name: Countyname, Length: 1969, dtype: int64

In [56]:
data.tail(100)

,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
3231,3232,Out of CO,Colorado,,0,80008,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3232,3233,Unassigned,Colorado,,0,90008,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/900...,https://coronavirus.jhu.edu/static/media/dashb...,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3233,3234,Out of CT,Connecticut,,0,80009,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3234,3235,Unassigned,Connecticut,,0,90009,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/900...,https://coronavirus.jhu.edu/static/media/dashb...,684,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3235,3236,Out of DE,Delaware,,0,80010,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3236,3237,Unassigned,Delaware,,0,90010,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/900...,https://coronavirus.jhu.edu/static/media/dashb...,139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3237,3238,Out of DC,District of Columbia,,0,80011,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3238,3239,Unassigned,District of Columbia,,0,90011,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/900...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3239,3240,Out of FL,Florida,,0,80012,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/cou

In [57]:
data.shape

(3331, 88)

In [58]:
data[data.Countyname.str.contains("Out of")]

,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
3221,3222,Out of AL,Alabama,,0,80001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3223,3224,Out of AK,Alaska,,0,80002,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3225,3226,Out of AZ,Arizona,,0,80004,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3227,3228,Out of AR,Arkansas,,0,80005,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3229,3230,Out of CA,California,,0,80006,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3231,3232,Out of CO,Colorado,,0,80008,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3233,3234,Out of CT,Connecticut,,0,80009,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3235,3236,Out of DE,Delaware,,0,80010,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None
3237,3238,Out of DC,District of Columbia,,0,80011,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,01/04/2021 11:30:50,,,,https://bao.arcgis.com/covid-19/jhu/county/800...,https://coronavirus.jhu.edu/static/media/dashb...,0,0,0,0,0

In [59]:
data = data[~data.Countyname.str.contains("Out of")]

In [60]:
data.shape

(3279, 88)

In [61]:
data = data[~data.Countyname.str.contains("Unassigned")]

In [62]:
data.shape

(3227, 88)

In [63]:
data = data[~data.ST_Name.str.contains("Puerto Rico")]

In [64]:
data.shape

(3149, 88)

In [66]:
data.drop(data.tail(7).index, inplace=True)

In [67]:
data.shape

(3142, 88)

In [68]:
data.tail(10)

,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
3132,3133,Niobrara,Wyoming,WY,56,56027,1.418440,5904.52,83.752094,13.6,2.8,81.9737,0.976045,01/04/2021 11:30:50,Govt Directed Social Distancing,3/21/2020 1:34:19 PM,per dde designee,https://bao.arcgis.com/covid-19/jhu/county/560...,https://coronavirus.jhu.edu/static/media/dashb...,141,2,40,104,118,92,114,24,24,4,0,2388,292,36,51036,0,0,44875,438,0,204403,468,0,0,2448,2189,13,84,33,0,0,16,113,2278,13,96,33,0,12,16,2335,355,335,340,262,2448,113,950,206,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,61.538462,NaN,1.275847e+10,4.596336e+05,"POLYGON ((-104.05479 43.50307, -104.05484 43.4..."
3133,3134,Park,Wyoming,WY,56,56029,0.480538,7096.58,34.101760,10.2,4.4,94.5614,0.976045,01/04/2021 11:30:50,Govt Directed Social Distancing,3/21/2020 1:34:19 PM,per dde designee,https://bao.arcgis.com/covid-19/jhu/county/560...,https://coronavirus.jhu.edu/static/media/dashb...,2081,10,636,782,1058,1532,2228,50,50,8,4,29324,2921,681,58873,0,0,44875,438,0,204403,6236,46,0,29121,26501,45,165,167,3,47,611,1582,27485,47,328,167,3,422,669,27539,4920,3460,3410,2476,29121,1582,11095,3760,46.0,14.0,0.0,36.0,14.0,32.0,40.0,1.0,38.0,0.0,0.0,18.0,30.0,28.0,156.868094,36.703297,23.214286,3.549520e+10,1.008405e+06,"POLYGON ((-109.10323 45.00582, -109.08070 44.9..."
3134,3135,Platte,Wyoming,WY,56,56031,1.781737,5241.65,93.392482,10.6,3.8,86.9995,0.976045,01/04/2021 11:30:50,Govt Directed Social Distancing,3/21/2020 1:34:19 PM,per dde designee,https://bao.arcgis.com/covid-19/jhu/county/560...,https://coronavirus.jhu.edu/static/media/dashb...,449,8,149,299,410,594,581,25,25,4,2,8566,899,178,54165,0,0,44875,438,0,204403,2033,8,0,8673,7693,1,7,59,0,0,215,698,8223,1,7,59,0,127,256,7975,1456,919,842,858,8673,698,3423,1175,8.0,1.0,0.0,6.0,9.0,4.0,1.0,0.0,2.0,0.0,0.0,3.0,3.0,2.0,93.392482,17.391304,0.000000,9.946588e+09,4.276800e+05,"POLYGON ((-104.65564 42.60947, -104.65503 42.2..."
3135,3136,Sheridan,Wyoming,WY,56,56033,0.660707,8510.57,56.229947,8.7,4.0,93.6523,0.976045,01/04/2021 11:30:50,Govt Directed Social Distancing,3/21/2020 1:34:19 PM,per dde designee,https://bao.arcgis.com/covid-19/jhu/county/560...,https://coronavirus.jhu.edu/static/media/dashb...,2573,17,729,614,965,1597,1989,88,71,12,3,30233,2551,631,58307,0,0,44875,438,0,204403,5894,18,0,30012,27543,114,340,211,0,0,552,1252,28534,153,368,211,0,183,563,28760,5351,3458,3574,2308,30012,1252,11735,3586,18.0,12.0,0.0,22.0,14.0,16.0,28.0,5.0,26.0,0.0,0.0,6.0,6.0,19.0,59.537591,44.175824,44.155844,1.299414e+10,5.645227e+05,"POLYGON ((-107.08489 44.99660, -106.26371 44.9..."
3136,3137,Sublette,Wyoming,WY,56,56035,1.001669,6104.15,61.143381,6.8,4.2,119.7738,0.000000,01/04/2021 11:30:50,Govt Directed Social Distancing,4/2/2020 6:41:16 PM,per dde designee,https://bao.arcgis.com/covid-19/jhu/county/560...,https://coronavirus.jhu.edu/static/media/dashb...,599,6,0,0,0,0,0,0,0,0,0,9813,658,179,74570,0,0,0,0,0,204403,0,1,0,9951,8883,0,43,12,0,0,282,731,9578,0,43,12,0,0,318,9220,2027,988,1063,629,9951,731,4209,1035,1.0,1.0,0.0,5.0,2.0,4.0,4.0,0.0,3.0,0.0,0.0,4.0,8.0,2.0,10.190564,NaN,NaN,2.373293e+10,7.206829e+05,"POLYGON ((-109.75472 43.46320, -109.75446 43.3..."
3137,3138,Sweetwater,Wyoming,WY,56,56037,0.529801,701

In [69]:
pd.set_option('display.max_columns', None)
data.describe()

,OBJECTID,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3142.0,3142.0,3.142000e+03,3142.000000,3142.0,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3142.000000,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3.142000e+03,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,2423.000000,1552.000000,3.142000e+03,3.142000e+03
mean,1571.500000,1.743939,6760.259682,118.816695,15.157829,4.132018,89.415545,1.224631,6466.437619,110.640356,1912.348504,1823.839593,2615.483450,3712.013049,5029.675684,298.811903,254.153405,29.178867,7.380013,1.041271e+05,1.332028e+04,2003.960853,52777.610439,0.0,0.0,4.490002e+05,7264.952578,0.0,5.521402e+06,1.509336e+04,66.257479,0.430936,1.027699e+05,6.275658e+04,1.264033e+04,679.655952,5.527425e+03,167.123488,240.132400,2452.500000,1.830615e+04,7.476283e+04,1.302231e+04,859.030236,5.593428e+03,185.460853,5.025449e+03,3321.386696,8.446375e+04,1.940349e+04,1.384188e+04,1.418459e+04,6589.166773,1.027699e+05,1.830615e+04,3.966885e+04,9081.928390,66.257479,92.561744,50.589115,71.213240,73.141311,62.534373,51.361235,47.219287,70.608211,31.392425,62.663908,72.590070,60.490134,59.139083,51.641875,53.976173,65.560210,6.808623e+09,3.714886e+05
std,907.161599,1.124206,2812.775484,88.330724,6.130698,1.503802,20.137309,0.826561,23526.417249,428.321697,6072.158986,5578.067255,7750.287741,10867.919006,14845.034687,985.479124,862.311711,102.185808,23.534068,3.334863e+05,4.559631e+04,6969.042922,13909.834889,0.0,0.0,5.247868e+05,8947.089724,0.0,6.326614e+06,4.499390e+04,373.291300,2.676469,3.299077e+05,1.428384e+05,5.412784e+04,2964.764416,4.050308e+04,1891.062520,1298.756878,8822.911004,1.255435e+05,2.035483e+05,5.579374e+04,3575.451348,4.095595e+04,2031.235583,4.558106e+04,13169.477778,2.245652e+05,6.335624e+04,4.449492e+04,5.151178e+04,19296.905349,3.299077e+05,1.255435e+05,1.275598e+05,25601.056119,373.291300,439.088775,421.117158,324.191008,279.130099,308.559747,323.263797,333.769984,559.947611,161.076173,303.965862,370.067258,286.522454,291.281498,66.224198,34.142614,29.709178,5.556968e+10,7.397408e+05
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,2.044030e+05,0.000000e+00,-11.000000,-1.000000,7.500000e+01,1.600000e+01,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,1.600000e+01,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,7.300000e+01,4.000000e+00,2.000000e+00,8.000000e+00,3.000000,7.500000e+01,0.000000e+00,3.900000e+01,3.000000,-11.000000,-28.000000,-12.000000,-103.000000,-136.000000

In [79]:
data.columns

Index(['OBJECTID', 'Countyname', 'ST_Name', 'ST_Abbr', 'ST_ID', 'FIPS',
       'FatalityRa', 'Confirmedb', 'DeathsbyPo', 'PCTPOVALL_', 'Unemployme',
       'Med_HH_Inc', 'State_Fata', 'DateChecke', 'EM_type', 'EM_date',
       'EM_notes', 'url', 'Thumbnail', 'Confirmed', 'Deaths', 'Age_85',
       'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Beds_Licen',
       'Beds_Staff', 'Beds_ICU', 'Ventilator', 'POP_ESTIMA', 'POVALL_201',
       'Unemployed', 'Median_Hou', 'Recovered', 'Active', 'State_Conf',
       'State_Deat', 'State_Reco', 'State_Test', 'AgedPop', 'NewCases',
       'NewDeaths', 'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop',
       'AsianPop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'Wh_Alone',
       'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More',
       'Not_Hisp', 'Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_Over75',
       'Agetotal', 'NonHisp', 'Age_35_64', 'Age_65_74', 'Day_1', 'Day_2',
       'Day_3', 'Day_4', 'Day_5', 'Day_6'

In [80]:
# remove Day_1 to Day_14 columns
data.drop(columns=['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'Day_8', 'Day_9', 'Day_10', 'Day_11', 'Day_12', 'Day_13', 'Day_14'], inplace=True)

In [81]:
data.head()

,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,1,Autauga,Alabama,AL,01,01001,1.161440,7742.67,89.926440,13.8,3.6,118.9591,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4305,50,815,1026,1498,2440,2271,85,55,6,2,55601,7587,942,59338,0,0,371934,4873,0,1972204,8050,37,0,55200,41412,10475,159,568,5,41,1012,1528,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,55200,1528,22052,4711,66.545566,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,2,Baldwin,Alabama,AL,01,01003,1.201650,6450.73,77.515113,9.8,3.6,115.4508,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,14064,169,3949,4792,7373,11410,13141,386,362,51,8,218022,21069,3393,57588,0,0,371934,4873,0,1972204,40665,109,0,208107,172768,19529,1398,1668,9,410,2972,9353,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,208107,9353,82998,24551,49.994955,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,3,Barbour,Alabama,AL,01,01005,2.156863,6149.27,132.631325,30.9,5.2,68.9280,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1530,33,422,551,841,1305,1515,74,30,5,2,24881,6788,433,34382,0,0,371934,4873,0,1972204,4634,2,0,25782,11898,12199,63,85,1,86,344,1106,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,25782,1106,9864,2820,8.038262,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,4,Bibb,Alabama,AL,01,01007,2.444208,8401.79,205.357143,21.8,4.0,92.3478,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1882,46,427,488,624,842,1280,35,25,4,1,22400,4400,344,46064,0,0,371934,4873,0,1972204,3661,19,0,22527,16801,4974,8,37,0,0,160,547,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,22527,547,9044,2122,84.821429,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,5,Blount,Alabama,AL,01,01009,1.327434,8205.39,108.921162,13.2,3.5,101.0645,1.310179,01/04/2021 11:30:50,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4746,63,866,1459,1776,2802,3330,25,25,6,2,57840,7527,878,50412,0,0,371934,4873,0,1972204,10233,17,0,57645,50232,820,124,198,18,174,818,5261,55054,862,141,198,18,437,935,52384,11112,6906,6786,4101,57645,5261,22608,6132,29.391425,76.190476,92.857143,2.456058e+09,286306.840721,"POLYGON ((-86.44507 34.24954, -86.40902 34.205..."


In [83]:
data.drop(columns=['OBJECTID', 'ST_ID', 'PCTPOVALL_', 'Med_HH_Inc', 'State_Fata', 'DateChecke', 'url', 'Thumbnail', 'State_Conf', 'State_Deat', 'State_Reco', 'State_Test', 'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'NonHisp'], inplace=True)

## State level data were dropped as well as population data that was repeated

In [84]:
data.head()

,Countyname,ST_Name,ST_Abbr,FIPS,FatalityRa,Confirmedb,DeathsbyPo,Unemployme,EM_type,EM_date,EM_notes,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,AgedPop,NewCases,NewDeaths,AsianPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,Age_35_64,Age_65_74,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,Autauga,Alabama,AL,01001,1.161440,7742.67,89.926440,3.6,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,4305,50,815,1026,1498,2440,2271,85,55,6,2,55601,7587,942,59338,0,0,8050,37,0,568,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,55200,22052,4711,66.545566,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,Baldwin,Alabama,AL,01003,1.201650,6450.73,77.515113,3.6,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,14064,169,3949,4792,7373,11410,13141,386,362,51,8,218022,21069,3393,57588,0,0,40665,109,0,1668,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,208107,82998,24551,49.994955,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,Barbour,Alabama,AL,01005,2.156863,6149.27,132.631325,5.2,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,1530,33,422,551,841,1305,1515,74,30,5,2,24881,6788,433,34382,0,0,4634,2,0,85,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,25782,9864,2820,8.038262,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,Bibb,Alabama,AL,01007,2.444208,8401.79,205.357143,4.0,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,1882,46,427,488,624,842,1280,35,25,4,1,22400,4400,344,46064,0,0,3661,19,0,37,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,22527,9044,2122,84.821429,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,Blount,Alabama,AL,01009,1.327434,8205.39,108.921162,3.5,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,4746,63,866,1459,1776,2802,3330,25,25,6,2,57840,7527,878,50412,0,0,10233,17,0,198,55054,862,141,198,18,437,935,52384,11112,6906,6786,4101,57645,22608,6132,29.391425,76.190476,92.857143,2.456058e+09,286306.840721,"POLYGON ((-86.44507 34.24954, -86.40902 34.205..."


In [85]:
data.drop(columns=['Age_85', 'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Agetotal', 'AsianPop'], inplace=True)

In [86]:
data.head()

,Countyname,ST_Name,ST_Abbr,FIPS,FatalityRa,Confirmedb,DeathsbyPo,Unemployme,EM_type,EM_date,EM_notes,Confirmed,Deaths,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,AgedPop,NewCases,NewDeaths,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Age_35_64,Age_65_74,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,Autauga,Alabama,AL,01001,1.161440,7742.67,89.926440,3.6,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,4305,50,85,55,6,2,55601,7587,942,59338,0,0,8050,37,0,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,22052,4711,66.545566,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,Baldwin,Alabama,AL,01003,1.201650,6450.73,77.515113,3.6,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,14064,169,386,362,51,8,218022,21069,3393,57588,0,0,40665,109,0,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,82998,24551,49.994955,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,Barbour,Alabama,AL,01005,2.156863,6149.27,132.631325,5.2,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,1530,33,74,30,5,2,24881,6788,433,34382,0,0,4634,2,0,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,9864,2820,8.038262,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,Bibb,Alabama,AL,01007,2.444208,8401.79,205.357143,4.0,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,1882,46,35,25,4,1,22400,4400,344,46064,0,0,3661,19,0,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,9044,2122,84.821429,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,Blount,Alabama,AL,01009,1.327434,8205.39,108.921162,3.5,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,4746,63,25,25,6,2,57840,7527,878,50412,0,0,10233,17,0,55054,862,141,198,18,437,935,52384,11112,6906,6786,4101,22608,6132,29.391425,76.190476,92.857143,2.456058e+09,286306.840721,"POLYGON ((-86.44507 34.24954, -86.40902 34.205..."


- FatalityRa = County Fatality Rate
- Confirmedb = County Confirmed divided by County Population * 100,000
- DeathsbyPo = County Deaths divided by County Population * 100,000
- Unemployme = Unemployment Rate
- EM_type, EM_date, EM_notes - could be turned into a column about Emergency Declaration
- Confirmed = County Level Confirmed Cases
- Deaths = County Level Deaths
- Beds_Licen = Number of Licensed Beds 
- Beds_Staff = Number of Staffed Beds 
- Beds_ICU = Number of ICU Beds 
- Ventilator = Average Ventilation Used Per Hospital
- POP_ESTIMA = Total Population
- POVALL_201 = Poverty Rate
- Median_Hou - Median Household Income
- Recovered = Number of Recovered Cases
- Active = Number of Active Cases
- AgedPop = TOtal Population Aged 65 Plus
- NewCases = New cases since yesterday
- NewDeaths = Newdeaths since yesterday
- Wh_Alone, Bk_Alone, AI_Alone, As_Alone, NH_Alone, SO_Alone, Two_More, Not_Hisp = Population numbers of each group
- Age_? = Population numbers of age groups
- NewCasebyP = I think this is new case by percentage
- ICU_Occ, Inpat_Occ = Percent Occupied of ICUs and Inpatient Beds
- Shape_Area, Shape_Length = for mapping



## Masking Dataset

In [70]:
df = covidcast.signal('fb-survey', 'smoothed_wearing_mask', date(2020, 9, 8), date(2021, 1, 3), 'county')
df2 = covidcast.signal('fb-survey', 'smoothed_others_masked', date(2020, 9, 8), date(2021, 1, 3), 'county')

In [75]:
df.head()

,geo_value,signal,time_value,issue,lag,value,stderr,sample_size,geo_type,data_source
0,01000,smoothed_wearing_mask,2020-09-08,2020-12-09,92,87.874513,1.706219,366.0096,county,fb-survey
1,02000,smoothed_wearing_mask,2020-09-08,2020-12-09,92,78.712871,4.093374,100.0000,county,fb-survey
2,04000,smoothed_wearing_mask,2020-09-08,2020-12-09,92,75.423725,3.754089,131.5270,county,fb-survey
3,04013,smoothed_wearing_mask,2020-09-08,2020-12-09,92,90.151765,1.594534,349.1927,county,fb-survey
4,04019,smoothed_wearing_mask,2020-09-08,2020-12-09,92,88.463022,2.866823,124.1801,county,fb-survey


In [76]:
df.geo_value.value_counts()

16000    118
35000    118
28000    118
53033    118
06001    118
        ... 
17183      1
37191      1
18085      1
48099      1
39087      1
Name: geo_value, Length: 729, dtype: int64

In [73]:
df.describe()

,lag,value,stderr,sample_size
count,65690.000000,65690.000000,65690.000000,65690.000000
mean,35.810428,89.572060,1.873058,405.864995
std,29.161640,6.272864,0.884784,463.929858
min,0.000000,51.179474,0.256537,100.000000
25%,5.000000,86.207682,1.153020,148.000000
50%,30.000000,91.029322,1.735842,240.309700
75%,62.000000,94.294567,2.477027,466.412725
max,92.000000,99.677919,4.965824,4633.860900


In [77]:
df2.head()

,geo_value,signal,time_value,issue,lag,value,stderr,sample_size,geo_type,data_source
0,01000,smoothed_others_masked,2020-11-24,2020-12-09,15,63.732745,2.362805,414.0201,county,fb-survey
1,04000,smoothed_others_masked,2020-11-24,2020-12-09,15,69.457396,3.936373,136.9090,county,fb-survey
2,04013,smoothed_others_masked,2020-11-24,2020-12-09,15,82.923867,2.051720,336.3820,county,fb-survey
3,04019,smoothed_others_masked,2020-11-24,2020-12-09,15,84.678313,2.964860,147.5946,county,fb-survey
4,05000,smoothed_others_masked,2020-11-24,2020-12-09,15,64.576451,2.575665,344.8160,county,fb-survey


In [78]:
df2.geo_value.value_counts()

41000    41
16000    41
39000    41
06065    41
35000    41
         ..
39087     1
18085     1
37191     1
13063     1
54079     1
Name: geo_value, Length: 699, dtype: int64

In [74]:
df2.describe()

,lag,value,stderr,sample_size
count,23085.000000,23085.000000,23085.000000,23085.000000
mean,5.526446,81.764531,2.291651,409.139082
std,2.250465,12.298847,1.045768,472.249071
min,0.000000,24.554959,0.420651,100.000000
25%,5.000000,75.495299,1.428610,149.000000
50%,5.000000,85.424989,2.109543,237.125400
75%,5.000000,91.132181,3.087063,473.310700
max,15.000000,98.750000,4.992825,4329.492700
